# Advent of Code 2023

## Day 1

### Part 1

In [29]:
with open('inputs/day1.txt') as f:
    s = f.read()[:-1]

def parse(n):
    return int(n[0] + n[-1])

sum([parse([c for c in n if c.isnumeric()]) for n in s.split('\n')])

54634

### Part 2

In [52]:
numerals = ['1','2','3','4','5','6','7','8','9','one','two','three','four','five','six','seven','eight','nine']
numeral_map = {'one':'1',
               'two':'2',
               'three':'3',
               'four':'4',
               'five':'5',
               'six':'6',
               'seven':'7',
               'eight':'8',
               'nine':'9'}

def parse_numeral(n):
    return numeral_map[n] if n in numeral_map else n


def extract_number(s):
    indices_left = {n:s.find(n) for n in numerals if s.find(n) >= 0}
    indices_right = {n:s.rfind(n) for n in numerals if s.find(n) >= 0}
    leftmost = parse_numeral(min(indices_left,key=indices_left.get))
    rightmost = parse_numeral(max(indices_right,key=indices_right.get))
    return int(leftmost + rightmost)


sum([extract_number(n) for n in s.split('\n')])

53855